In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import zipfile
import tempfile
import math
import pathlib

In [ ]:
from google.colab import drive
drive.mount ('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/drive/My Drive/sign_mnist_train.csv')
test = pd.read_csv('/drive/My Drive/sign_mnist_test.csv')

In [ ]:
x_train = train.drop(['pixel1'], axis=1).values
y_train = train['label'].values
x_test = test.drop(['pixel1'], axis=1).values
y_test = test['label'].values
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [ ]:
model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(28, 28, 1)),
        keras.layers.Reshape(target_shape=(28, 28, 1)),
        keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation=tf.nn.relu, padding='same'),
        
        keras.layers.Conv2D(filters=32, kernel_size=(6,6), activation=tf.nn.relu, padding='same'),
        
        keras.layers.Conv2D(filters=64, kernel_size=(12,12), activation=tf.nn.relu, padding='same'),

        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(1024, activation='relu'),
        keras.layers.Dropout(0.20),
        keras.layers.Dense(25, activation='softmax')                  
])

In [ ]:
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10,validation_data=(x_test, y_test))

Epoch 1/10
858/858 [==============================] - 7s 8ms/step - loss: 0.5174 - accuracy: 0.8402 - val_loss: 0.2403 - val_accuracy: 0.9269
Epoch 2/10
858/858 [==============================] - 7s 8ms/step - loss: 0.0101 - accuracy: 0.9971 - val_loss: 0.1961 - val_accuracy: 0.9426
Epoch 3/10
858/858 [==============================] - 7s 8ms/step - loss: 0.0280 - accuracy: 0.9921 - val_loss: 0.4933 - val_accuracy: 0.8970
Epoch 4/10
858/858 [==============================] - 7s 8ms/step - loss: 0.0085 - accuracy: 0.9978 - val_loss: 0.3009 - val_accuracy: 0.9387
Epoch 5/10
858/858 [==============================] - 7s 8ms/step - loss: 0.0053 - accuracy: 0.9984 - val_loss: 0.3442 - val_accuracy: 0.9416
Epoch 6/10
858/858 [==============================] - 7s 8ms/step - loss: 0.0068 - accuracy: 0.9986 - val_loss: 0.3380 - val_accuracy: 0.9420
Epoch 7/10
858/858 [==============================] - 7s 8ms/step - loss: 1.8535e-05 - accuracy: 1.0000 - val_loss: 0.3313 - val_accuracy: 0.9463
Ep

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 64)        294976    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)             

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

225/225 - 1s - loss: 0.4006 - accuracy: 0.9240

Test accuracy: 0.9240100383758545


In [ ]:
pip install -q tensorflow-model-optimization

In [ ]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_2 (QuantizeLa (None, 28, 28, 1)         3         
_________________________________________________________________
quant_reshape_1 (QuantizeWra (None, 28, 28, 1)         1         
_________________________________________________________________
quant_conv2d_3 (QuantizeWrap (None, 28, 28, 16)        195       
_________________________________________________________________
quant_conv2d_4 (QuantizeWrap (None, 28, 28, 32)        18531     
_________________________________________________________________
quant_conv2d_5 (QuantizeWrap (None, 28, 28, 64)        295107    
_________________________________________________________________
quant_max_pooling2d_1 (Quant (None, 14, 14, 64)        1         
_________________________________________________________________
quant_flatten_1 (QuantizeWra (None, 12544)            

In [ ]:
train_images_subset = x_train[0:1000] # out of 60000
train_labels_subset = y_train[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


2/2 [==============================] - 1s 268ms/step - loss: 0.2571 - accuracy: 0.9011 - val_loss: 0.0811 - val_accuracy: 1.0000


In [ ]:
_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   x_test, y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9240100383758545
Quant test accuracy: 0.8825989961624146


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpbpl8_s7r/assets


INFO:tensorflow:Assets written to: /tmp/tmpbpl8_s7r/assets


In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(x_test):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == y_test).mean()
  return accuracy

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.


Quant TFLite test_accuracy: 0.8830172894590073
Quant TF test accuracy: 0.8825989961624146
